In [1]:
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from pathlib import Path
from collections import defaultdict

/media/data/inno_courses/DLS/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = SentenceTransformer('cointegrated/rubert-tiny2')

In [3]:
index = faiss.read_index("embeddings/embeddings.index")  # read index from file

In [4]:
embeddings = np.load("embeddings/embeddings.npy", allow_pickle=True)

In [5]:
embeddings.shape

(49727, 312)

In [6]:
chunk_files = sorted(Path('.').glob('dataset/preproc_data*.parquet'))
dfs = [pd.read_parquet(f) for f in chunk_files]
df = pd.concat(dfs, ignore_index=True)

# Ranking metrics functions

In [11]:
def precision_at_k(relevant, k):
    return np.sum(relevant[:k]) / k

def recall_at_k(relevant, total_relevant, k):
    if total_relevant == 0:
        return 0.0
    return np.sum(relevant[:k]) / total_relevant


def hits_at_k(relevant, k):
    return 1.0 if np.sum(relevant[:k]) > 0 else 0.0

def mrr(relevant):
    for idx, rel in enumerate(relevant, 1):
        if rel:
            return 1.0 / idx
    return 0.0

def dcg(relevant, k):
    return np.sum(relevant[:k] / np.log2(np.arange(2, k + 2)))

def ndcg_at_k(relevant, k):
    ideal_relevant = np.sort(relevant[::-1])
    idcg = dcg(ideal_relevant, k)
    if idcg == 0:
        return 0.0
    return dcg(relevant, k) / idcg

def average_precision_at_k(relevant, k):
    hits = 0
    sum_precisions = 0.0
    for i in range(k):
        if relevant[i]:
            hits += 1
            sum_precisions += hits / (i + 1)
    if hits == 0:
        return 0.0
    return sum_precisions / hits

In [15]:
def evaluate_index(index, embeddings, df, k=10, n_eval=100):
    precisions = []
    recalls = []
    hits = []
    mrrs = []
    ndcgs = []
    aps = []

    class_counts = df['classifierByIPS'].value_counts().to_dict()

    for _ in range(n_eval):
        # choose random query from embedding
        i = np.random.randint(0, len(embeddings))
        query = embeddings[i].reshape(1, -1)
        query_class = df.iloc[i]['classifierByIPS']

        if not isinstance(query_class, str) or query_class == "UNKNOWN":
            continue

        _, topk = index.search(query, k+1)
        topk = topk[0][1:]  # except the same one

        topk_classes = df.iloc[topk]['classifierByIPS'].values
        relevant = (topk_classes == query_class).astype(int)

        precisions.append(precision_at_k(relevant, k))
        total_relevant = class_counts.get(query_class, 0) - 1
        total_relevant = max(total_relevant, 0)
        recalls.append(recall_at_k(relevant, total_relevant, k))
        hits.append(hits_at_k(relevant, k))
        mrrs.append(mrr(relevant))
        ndcgs.append(ndcg_at_k(relevant, k))
        aps.append(average_precision_at_k(relevant, k))

    print("Evaluation results:")
    print(f"Precision@{k}: {np.mean(precisions):.3f}")
    print(f"Recall@{k}: {np.mean(recalls):.3f}")
    print(f"Hits@{k}:     {np.mean(hits):.3f}")
    print(f"MRR:          {np.mean(mrrs):.3f}")
    print(f"NDCG@{k}:     {np.mean(ndcgs):.3f}")
    print(f"MAP@{k}:      {np.mean(aps):.3f}")

In [ ]:
evaluate_index(index, embeddings, df, 10, 100)

Evaluation results:
Precision@10: 0.107
Recall@10: 0.034
Hits@10:     0.259
MRR:          0.214
NDCG@10:     0.509
MAP@10:      0.196


: 

In [ ]:
import time

def measure_faiss_speed(index, embeddings, n_queries=100):
    total_time = 0.0
    for _ in range(n_queries):
        i = np.random.randint(0, len(embeddings))
        query = embeddings[i].reshape(1, -1)
        start = time.time()
        _ = index.search(query, 10)
        total_time += time.time() - start
    avg_time_ms = (total_time / n_queries) * 1000
    return avg_time_ms
measure_faiss_speed(index, embeddings)

7.3235719203948975

In [ ]:
# Real query test

In [13]:
df[['classifier_code', 'classifier_name']] = df['classifierByIPS'].str.split('$', n=1, expand=True)


In [15]:
df['classifier_level2'] = df['classifier_code'].str.extract(r'^(\d{3}\.\d{3})')

In [16]:
print(df['classifier_level2'].value_counts())

classifier_level2
010.140    17889
010.070     5480
210.010     2896
210.020     2102
020.010     1997
           ...  
140.030        1
100.030        1
090.020        1
070.020        1
050.050        1
Name: count, Length: 158, dtype: int64


In [21]:
df.columns

Index(['pravogovruNd', 'issuedByIPS', 'docdateIPS', 'docNumberIPS',
       'doc_typeIPS', 'headingIPS', 'doc_author_normal_formIPS', 'signedIPS',
       'statusIPS', 'actual_datetimeIPS', 'actual_datetime_humanIPS',
       'is_widely_used', 'textIPS', 'classifierByIPS', 'keywordsByIPS',
       'text_clean', 'tokens', 'lemmatized_text', 'classifier_code',
       'classifier_name', 'classifier_level2'],
      dtype='object')

In [ ]:
query = "украли кошелек"
query_vec = model.encode([query]) # encode
query_vec = query_vec / np.linalg.norm(query_vec) # normalyze

faiss.normalize_L2(query_vec)

k = 50
# find the closest
similarities, indices = index.search(query_vec, k)

# filter resilt from top 50 to top 5
results = []
for idx, sim in zip(indices[0], similarities[0]):
    if idx == -1:  # if idx not exist, skip
        continue
    
    row = df.iloc[idx]
    classifier = str(row['classifier_level2']).strip() if pd.notna(row['classifier_level2']) else "UNKNOWN"
    
    # add if not UNKNOWN
    if classifier != "UNKNOWN":
        results.append({
            'index': idx,
            'similarity': sim,
            'classifier': row['classifier_code'],
            'classifier_level2': classifier,
            'text': row['textIPS'],
            'heading': row['headingIPS']
        })

# group by classifier_level2 (2 numbers from classifier_code)
grouped = defaultdict(list)
for res in results:
    grouped[res['classifier_level2']].append(res)

top_results = []
# take from each grouped 1 with max similarity
if len(grouped) >= 5:
    for cls in sorted(grouped.keys(), key=lambda x: max(y['similarity'] for y in grouped[x]), reverse=True)[:5]:
        best_in_cls = max(grouped[cls], key=lambda x: x['similarity'])
        top_results.append(best_in_cls)

# take from each grouped 2 with max similarity
else:
    candidates = []
    for cls, items in grouped.items():
        items_sorted = sorted(items, key=lambda x: -x['similarity'])
        candidates.extend(items_sorted[:2])
    # return only top 5
    top_results = sorted(candidates, key=lambda x: -x['similarity'])[:5]

print("Топ-5 результатов с учетом классов:")
for i, res in enumerate(top_results, 1):
    print(f"{i}. Схожесть: {res['similarity']:.3f} | Класс: {res['classifier_level2']}")
    print(f"Текст: {res['text']}")

Топ-5 результатов с учетом классов:
1. Схожесть: 0.474 | Класс: 010.070
Текст:  
 ПРАВИТЕЛЬСТВО РСФСР 
 РАСПОРЯЖЕНИЕ 
 от 28 декабря 1991 г. N 239-р
 г. Москва 
1. Министерству экономики и финансов РСФСР отпустить в 1992-1993 годах Внешторгбанку РСФСР для продажи на экспорт 1,3 тонны золота в счет сверхплановой добычи производственным объединением "Лензолото" в 1991-1992 годах.
2. Внешторгбанку РСФСР перечислить средства, вырученные от реализации 1,3 тонны золота, в распоряжение администрации Иркутской области для закупки продовольствия, товаров первой необходимости и технологий по переработке сельскохозяйственной продукции.
3. Администрации Иркутской области возместить Внешторгбанку РСФСР стоимость проданного золота в советских рублях по расчетным ценам, действующим при сдаче золота в Государственный фонд драгоценных металлов и драгоценных камней РСФСР. 
Первый заместитель Председателя
Правительства Российской Федерации Г. Бурбулис 
 
2. Схожесть: 0.472 | Класс: 030.050
Текст:  
 ПРАВ